<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=46ea99dc0defd0f86fb066b68120c5f570d91b139779575e3bc2c68c460872cf
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-30 14:56:50
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.42 C
-------------------
Today PnL: -41.32 K (-0.29%)
Current PnL: -27.06 L (-17.25%)
CY Booked + Current PnL: -12.77 L (-8.14%)
-------------------
Total profit:  1.40 L
Total loss:  -28.46 L
-------------------
Total Booked + Current PnL: 14.21 L (10.95%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.06%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 92.43 L (65.1%)
Deployed:  1.30 C
Current:  1.42 C
CAGR/XIRR %: 4.96%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,-0.67,-20.06,25.20,0.09,20302.0,-20212.0,80565.0,86.98,35.0,M-SC,3.04,253.0,-1.00,0.57,5.84,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,-0.13,12.69,7.72,21.40,23092.0,33696.0,299122.0,4.55,64.0,X-LC,5.29,12.0,1.46,2.11,19.83,XY25,NTT,FMCG
51,MASFIN,398.61,0.08,-4.23,27.43,22.05,25740.0,-4140.0,93840.0,-17.34,47.0,H-SC,6.49,164.0,-0.16,0.66,36.40,XR,ATH,FINANCE
84,VOLTAS,1530.00,-1.91,5.90,13.02,19.69,26438.0,11313.0,203055.0,-4.39,45.0,X-MC,2.27,78.0,0.43,1.43,13.10,XY25,NTT,AC
44,ITC,452.00,-0.45,-1.77,12.77,10.78,30146.0,-4247.0,236071.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.66,3.38,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,15.97,-5.57,55.63,46.95,65818.0,-6983.0,118314.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.83,48.22,MH,ATH,POWER
62,REPCOHOME,880.00,2.81,-10.80,112.79,89.82,288262.0,-30931.0,255574.0,-55.29,57.0,H-SC,5.36,160.0,-0.11,1.80,35.22,XY24,NTT,FINANCE
39,INDIAMART,4810.62,2.61,-3.02,113.16,106.72,135351.0,-3726.0,119610.0,-51.96,47.0,H-SC,5.29,139.0,-0.03,0.84,19.65,AR,ATH,MISC
82,VALIANTORG,838.00,1.79,-43.21,206.68,74.15,192016.0,-70700.0,92905.0,-267.19,51.0,H-SC,21.43,149.0,-0.37,0.65,19.09,XR,NTT,CHEMICALS
21,DABUR,735.00,1.37,-3.03,48.19,43.70,115926.0,-7508.0,240560.0,-11.07,45.0,X-MC,1.98,72.0,-0.06,1.69,11.57,XY24,BTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.00,-5.00,-62.90,169.72,0.07,86376.0,-86284.0,50893.0,1764.90,46.0,L-SC,4.55,268.0,-1.00,0.36,26.57,OX40N,NTT,JEWELLERY
45,JCHAC,2282.00,-4.24,-37.58,60.20,-0.01,48023.0,-48033.0,79772.0,16226.09,35.0,M-SC,13.03,233.0,-1.00,0.56,2.10,OX40N,NTT,AC
30,HAPPSTMNDS,1480.71,-3.73,-43.19,225.36,84.85,164098.0,-55349.0,72816.0,-32.49,28.0,H-SC,13.70,147.0,-0.34,0.51,0.00,AR,ATH,IT
13,BERGEPAINT,680.00,-3.60,-6.47,29.52,21.14,62767.0,-14714.0,212625.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.50,18.08,XY24,NTT,PAINTS
28,GLAXO,3466.20,-3.09,-3.62,44.38,39.15,85238.0,-7220.0,192064.0,-22.56,29.0,X-MC,10.65,60.0,-0.08,1.35,24.33,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.16,-0.73,83.01,81.67,77304.0,-684.0,93126.0,8388.89,67.0,L-SC,15.93,271.0,-0.01,0.66,80.34,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-1.03,-6.96,123.48,107.91,169613.0,-10283.0,137361.0,-24.20,38.0,M-SC,11.62,216.0,-0.06,0.97,2.42,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,0.04,-19.92,24.90,0.02,34806.0,-34774.0,139785.0,113.17,33.0,M-SC,9.00,234.0,-1.00,0.98,21.59,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,-0.67,-20.06,25.20,0.09,20302.0,-20212.0,80565.0,86.98,35.0,M-SC,3.04,253.0,-1.00,0.57,5.84,OX40N,NTT,DURABLES
48,KANSAINER,340.00,0.56,-21.30,44.19,13.47,93780.0,-57447.0,212220.0,-67.73,55.0,H-SC,6.75,158.0,-0.61,1.49,7.91,XY24,NTT,PAINTS
68,SIS,528.00,-0.79,-23.40,58.30,21.26,49587.0,-25977.0,85055.0,2007.77,52.0,H-SC,4.60,166.0,-0.52,0.60,14.92,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-0.16,8.21,59.2,72.27,108546.0,13907.0,183355.0,-6.99,58.0,M-LC,5.06,99.0,0.13,1.29,14.89,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.16,8.21,59.20,72.27,108546.0,13907.0,183355.0,-6.99,58.0,M-LC,5.06,99.0,0.13,1.29,14.89,XR,NTT,IT
6,ASIANTILES,137.00,0.16,-0.73,83.01,81.67,77304.0,-684.0,93126.0,8388.89,67.0,L-SC,15.93,271.0,-0.01,0.66,80.34,XR,NTT,CERAMICS
39,INDIAMART,4810.62,2.61,-3.02,113.16,106.72,135351.0,-3726.0,119610.0,-51.96,47.0,H-SC,5.29,139.0,-0.03,0.84,19.65,AR,ATH,MISC
26,FINCABLES,1641.55,-1.03,-6.96,123.48,107.91,169613.0,-10283.0,137361.0,-24.20,38.0,M-SC,11.62,216.0,-0.06,0.97,2.42,OX40N,ATH,CABLES
49,KPIGREEN,730.66,15.97,-5.57,55.63,46.95,65818.0,-6983.0,118314.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.83,48.22,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DIXON,18931.72,-0.92,-21.77,61.15,26.07,93391.0,-42497.0,152724.0,-61.93,23.0,X-MC,15.49,56.0,-0.46,1.08,0.00,X40N,ATH,IT
9,BAJAJHFL,181.50,-0.14,-19.45,93.00,55.47,182260.0,-47307.0,195979.0,-28.68,26.0,X-MC,16.99,64.0,-0.26,1.38,0.42,X40N,ATH,FINANCE
35,HONAUT,58357.33,-0.20,-19.14,77.24,43.32,101725.0,-31172.0,131700.0,-30.52,28.0,X-SC,6.93,90.0,-0.31,0.93,1.12,X40N,ATH,ELECTRICAL
28,GLAXO,3466.20,-3.09,-3.62,44.38,39.15,85238.0,-7220.0,192064.0,-22.56,29.0,X-MC,10.65,60.0,-0.08,1.35,24.33,X40,ATH,PHARMA
54,PAGEIND,51605.08,-2.33,-11.97,45.53,28.11,80725.0,-24110.0,177300.0,-36.23,29.0,X-MC,15.05,55.0,-0.30,1.25,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,680.0,-3.60,-6.47,29.52,21.14,62767.0,-14714.0,212625.0,-13.02,36.0,X-MC,1.43,74.0,-0.23,1.50,18.08,XY24,NTT,PAINTS
67,SIEMENS,4671.5,-1.05,-18.80,54.58,25.51,82473.0,-34990.0,151105.0,-7.31,35.0,H-LC,1.44,49.0,-0.42,1.06,11.35,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,-0.59,4.41,15.67,20.77,38308.0,10320.0,244464.0,-4.74,49.0,X-MC,1.64,62.0,0.27,1.72,11.63,X40N,NTT,BREWERIES
21,DABUR,735.0,1.37,-3.03,48.19,43.70,115926.0,-7508.0,240560.0,-11.07,45.0,X-MC,1.98,72.0,-0.06,1.69,11.57,XY24,BTT,FMCG
44,ITC,452.0,-0.45,-1.77,12.77,10.78,30146.0,-4247.0,236071.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.66,3.38,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-0.19,-39.43,123.20,35.20,96023.0,-50729.0,77941.0,2.24,32.0,X-SC,17.34,92.0,-0.53,0.55,0.00,X40,NTT,FOOTWEAR
20,COLPAL,3726.84,-0.76,-21.99,81.41,41.51,167249.0,-57925.0,205440.0,-7.19,30.0,X-MC,8.40,61.0,-0.35,1.45,0.00,XY25,ATH,FMCG
23,DIXON,18931.72,-0.92,-21.77,61.15,26.07,93391.0,-42497.0,152724.0,-61.93,23.0,X-MC,15.49,56.0,-0.46,1.08,0.00,X40N,ATH,IT
54,PAGEIND,51605.08,-2.33,-11.97,45.53,28.11,80725.0,-24110.0,177300.0,-36.23,29.0,X-MC,15.05,55.0,-0.30,1.25,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-0.14,-19.45,93.00,55.47,182260.0,-47307.0,195979.0,-28.68,26.0,X-MC,16.99,64.0,-0.26,1.38,0.42,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.0,0.39,-28.84,100.74,42.84,46599.0,-18749.0,46257.0,-53.17,47.0,X-SC,32.04,83.0,-0.40,0.33,4.65,XY24,NTT,MISC
60,RELAXO,1176.0,-0.57,-48.07,193.27,52.31,145702.0,-69772.0,75388.0,-44.73,41.0,X-SC,7.35,91.0,-0.48,0.53,0.91,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.0,-0.19,-39.43,123.20,35.20,96023.0,-50729.0,77941.0,2.24,32.0,X-SC,17.34,92.0,-0.53,0.55,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.0,0.10,-3.59,28.69,24.08,35116.0,-4552.0,122398.0,-11.42,39.0,X-SC,7.48,89.0,-0.13,0.86,15.33,XY24,NTT,HEALTHCARE
61,RELIANCE,1952.0,-0.10,-0.40,26.42,25.92,34674.0,-522.0,131240.0,-4.24,51.0,X-LC,9.23,17.0,-0.02,0.92,33.42,XY25,BTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-0.04,-10.63,35.06,20.71,118514.0,-40206.0,338031.0,-22.98,55.0,X-LC,3.10,1.0,-0.34,2.38,12.95,X40,ATH,IT
42,INFY,1972.00,-0.81,11.06,20.88,34.24,72555.0,34600.0,347488.0,-13.50,57.0,X-LC,7.29,2.0,0.48,2.45,20.26,X40,NTT,IT
77,TMPV,600.00,0.61,-26.05,66.30,22.98,110754.0,-58848.0,167050.0,-23.08,51.0,X-LC,6.17,3.0,-0.53,1.18,5.07,XY24,NTT,AUTO
83,VBL,671.64,0.56,-1.95,38.37,35.67,118826.0,-6157.0,309685.0,-13.58,59.0,X-LC,3.03,4.0,-0.05,2.18,11.63,X40N,ATH,FMCG
44,ITC,452.00,-0.45,-1.77,12.77,10.78,30146.0,-4247.0,236071.0,-41.56,44.0,X-LC,2.02,5.0,-0.14,1.66,3.38,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.0,0.43,-37.75,113.47,32.89,54536.0,-29144.0,48062.0,-693.69,51.0,L-MC,6.72,259.0,-0.53,0.34,32.36,XR,NTT,BANKS
6,ASIANTILES,137.0,0.16,-0.73,83.01,81.67,77304.0,-684.0,93126.0,8388.89,67.0,L-SC,15.93,271.0,-0.01,0.66,80.34,XR,NTT,CERAMICS
14,BSOFT,831.7,-2.21,-22.60,94.82,50.79,103221.0,-31793.0,108860.0,-0.66,51.0,H-SC,11.06,151.0,-0.31,0.77,26.75,XR,ATH,IT
61,RELIANCE,1952.0,-0.10,-0.40,26.42,25.92,34674.0,-522.0,131240.0,-4.24,51.0,X-LC,9.23,17.0,-0.02,0.92,33.42,XY25,BTT,REFINERIES
71,SURYODAY,216.0,0.07,-20.34,54.44,23.03,77662.0,-36414.0,142657.0,55.70,52.0,H-SC,6.84,167.0,-0.47,1.00,41.54,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.16,-0.73,83.01,81.67,77304.0,-684.0,93126.0,8388.89,67.0,L-SC,15.93,271.0,-0.01,0.66,80.34,XR,NTT,CERAMICS
14,BSOFT,831.70,-2.21,-22.60,94.82,50.79,103221.0,-31793.0,108860.0,-0.66,51.0,H-SC,11.06,151.0,-0.31,0.77,26.75,XR,ATH,IT
49,KPIGREEN,730.66,15.97,-5.57,55.63,46.95,65818.0,-6983.0,118314.0,-12.86,62.0,H-SC,9.78,154.0,-0.11,0.83,48.22,MH,ATH,POWER
76,TITAGARH,1548.00,-1.91,-16.18,73.87,45.74,149947.0,-39189.0,202988.0,11.66,65.0,H-SC,5.26,174.0,-0.26,1.43,32.02,XY24,NTT,ENGINEERING
86,WIPRO,420.00,-0.16,8.21,59.20,72.27,108546.0,13907.0,183355.0,-6.99,58.0,M-LC,5.06,99.0,0.13,1.29,14.89,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.06
1,25,44.18
2,50,75.13


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.21
MC    30.09
LC    25.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.47
X40      23.57
X40N     12.63
XY25      9.92
XR        9.64
AR        8.61
OX40N     7.34
SR        0.97
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.69
H-SC    23.55
X-LC    22.29
M-SC    11.23
X-SC     7.90
H-MC     4.70
L-SC     1.53
M-MC     1.36
M-LC     1.29
H-LC     1.06
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.11,-7.40,41.94
IT,12.59,-17.64,76.79
FINANCE,10.43,-16.53,66.00
MISC,7.04,-30.10,83.33
ELECTRICAL,5.81,-13.76,55.43
PAINTS,5.68,-14.32,31.30
INSURANCE,4.67,-2.19,37.16
PHARMA,3.97,-4.10,36.86
AUTO,2.81,-31.46,77.04


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162708.0,21
AR,1347810.0,10
XR,1293627.0,13
X40,1049907.0,15
X40N,891859.0,9
OX40N,750117.0,10
XY25,399013.0,7
SR,282486.0,2
MH,65818.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3633489.0,25
X-MC,1464327.0,16
M-SC,1463099.0,15
X-LC,899252.0,13
X-SC,798412.0,8
H-MC,410367.0,3
L-SC,244990.0,3
M-LC,108546.0,1
H-LC,82473.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1242483.0      6
           AR         918195.0      5
M-SC       XY24       824432.0      6
H-SC       XR         791804.0      7
X-MC       X40        498111.0      7
           XY24       415054.0      3
X-LC       X40        369551.0      6
X-MC       X40N       357475.0      4
H-SC       OX40N      332703.0      4
M-SC       OX40N      331038.0      5
X-SC       X40N       318319.0      3
           XY24       297848.0      3
H-SC       SR         282486.0      2
H-MC       AR         219640.0      2
X-LC       X40N       216065.0      2
X-MC       XY25       193687.0      2
X-LC       XY24       192164.0      2
H-MC       XY24       190727.0      1
X-SC       X40        182245.0      2
M-SC       XR         180127.0      2
L-SC       XR         158614.0      2
M-SC       AR         127502.0      2
X-LC       XY25       121472.0      3
M-LC       XR         108546.0      1
L-SC       OX40N       86376.0      1
H-LC       AR          82473.0      1
H-SC       MH          65818.0      1
L-MC       XR          54536.0      1
M-MC       XY25        51722.0      1
L-LC       XY25        32132.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
